<a href="https://colab.research.google.com/github/sbutori/autoestudo-ccm10s06/blob/main/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Treinamento: Classificação de Imagens (CIFAR-10)

Esse notebook treina o modelo e gera um arquivo `cifar_net.pth` ao final, que pode ser usado por nossa API em Flask.

## Importar bibliotecas

In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


## Definir transformações para data augmentation

In [2]:
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, padding=4),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

## Carregar dados CIFAR-10

In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:03<00:00, 43778199.49it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Definir a rede neural

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()


## Definir função de perda e otimizador

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Treinamento

In [8]:
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print(f'[Época {epoch + 1}, Perda do Batch {i + 1}]: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Treinamento Encerrado')
torch.save(net.state_dict(), 'cifar_net.pth')

[Época 1, Perda do Batch 100]: 2.307
[Época 1, Perda do Batch 200]: 2.304
[Época 1, Perda do Batch 300]: 2.303
[Época 1, Perda do Batch 400]: 2.303
[Época 1, Perda do Batch 500]: 2.302
[Época 2, Perda do Batch 100]: 2.302
[Época 2, Perda do Batch 200]: 2.300
[Época 2, Perda do Batch 300]: 2.298
[Época 2, Perda do Batch 400]: 2.294
[Época 2, Perda do Batch 500]: 2.288
[Época 3, Perda do Batch 100]: 2.277
[Época 3, Perda do Batch 200]: 2.264
[Época 3, Perda do Batch 300]: 2.246
[Época 3, Perda do Batch 400]: 2.236
[Época 3, Perda do Batch 500]: 2.211
[Época 4, Perda do Batch 100]: 2.194
[Época 4, Perda do Batch 200]: 2.176
[Época 4, Perda do Batch 300]: 2.160
[Época 4, Perda do Batch 400]: 2.150
[Época 4, Perda do Batch 500]: 2.138
[Época 5, Perda do Batch 100]: 2.123
[Época 5, Perda do Batch 200]: 2.118
[Época 5, Perda do Batch 300]: 2.104
[Época 5, Perda do Batch 400]: 2.081
[Época 5, Perda do Batch 500]: 2.068
[Época 6, Perda do Batch 100]: 2.055
[Época 6, Perda do Batch 200]: 2.039
[